In [45]:
from lxml import etree
import sqlite3

In [46]:
xml_db = 'wordnet/data/xml_synsets.db'

In [47]:
tree = etree.parse("estwn-et-2.0.0.beta.xml")

In [48]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)

def sqlTables(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS synset_data(

                                        id TEXT NOT NULL,
                                        status TEXT NOT NULL,
                                        confidence_score TEXT NOT NULL

                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")

In [190]:

def fetch_data(tree):
    synsets = tree.xpath('/LexicalResource/Lexicon/Synset')
    for sset in synsets:
        id_list.append(sset.attrib['id'])
        status_list.append(sset.attrib['status'])
        conf_list.append(sset.attrib['confidenceScore'])
        '''
        for relations
        
        if len(sset) > 2:
            for data in sset:
                #print(data.attrib)
                
                status_list.append(data.attrib['target'])
                conf_list.append((data.attrib['confidenceScore']))
                source_sense.append(data[0].attrib['sourceSense'])           
                '''

In [50]:
def upload_data(db_file):
    sqlTables(db_file)
    conn = create_connection(db_file)
    cursor = conn.cursor()
    
    with conn:
        for i in range(len(id_list)):
            sset_id = id_list[i]
            status = status_list[i]
            conf = conf_list[i]  
            
            cursor.execute("INSERT INTO synset_data(id, status, confidence_score) VALUES(?,?,?)"\
                                                        ,(sset_id, status, conf))
            conn.commit()

In [191]:
id_list = []
status_list=[]
conf_list=[]
source_sense=[]

fetch_data(tree)

KeyError: 'status'

In [188]:
upload_data(xml_db)

Get all lexical entries

In [51]:
entries = tree.xpath('/LexicalResource/Lexicon/LexicalEntry')
print(len(entries))

113840


In [209]:
entry = entries[4]
print(entry[1].attrib['synset'])

estwn-et-39243-n


Each lexicon element is al lemma and pos pair and it can correspond to many synsets

In [59]:
entry = entries[0]
print(entry[0].tag)
print(entry[0].attrib['writtenForm'])
print(entry[0].attrib['partOfSpeech'])
entry[1].attrib['synset']

Lemma
röntgen
n


'estwn-et-41417-n'

In [53]:
subset = entries[104]
print(len(subset))
print(subset[2].tag)
print(subset[0].attrib['writtenForm'])
print(subset[6].attrib)

7
Sense
kaotaja
{'synset': 'estwn-et-13993-n', 'id': 's-kaotaja-n2', 'note': '===\nSünonüümisõnastiku sünonüümirea number: 2817\n', 'status': 'unchecked'}


Corresponding synset mapping is defined as sense element in the second, third,... components of an entry

In [66]:
print(entry[1].tag)
print('Long name = ', entry[1].attrib['id'])
print('Status = ', entry[1].attrib['status'])
print('synset =', entry[1].attrib['synset'])


Sense
Long name =  s-röntgen-n1
Status =  unchecked
synset = estwn-et-41417-n


In [ ]:
There can be several sense components

In [65]:
print(len(entries[2]))
print(entries[2][1].attrib)
print(entries[2][2].attrib)

3
{'synset': 'estwn-et-3972-v', 'id': 's-sisse_ajama-v1', 'status': 'unchecked'}
{'synset': 'estwn-et-19302-v', 'id': 's-sisse_ajama-v2', 'status': 'unchecked'}


Get all sysets

In [60]:
synsets = tree.xpath('/LexicalResource/Lexicon/Synset')
print(len(synsets))

83557


In [64]:
sset = synsets[105]
print(len(sset))
print(sset[0].attrib)

2
{'language': 'et', 'sourceSense': 's-rõõmupisar-n1'}


The most important property of a synset is its identifier, status and confidence 

In [1]:
synset=synsets[0]
print('synset_id=', synset.attrib['id'])
print('synset_status=', synset.attrib['status'])
print('confidence_score=', synset.attrib['confidenceScore'])


NameError: name 'synsets' is not defined

The first element of the synset chhild is its definition that links back to lexicon element senses

In [63]:
print(synset[0].tag)
print('language = ', synset[0].attrib['language'])
print('lecicon_sense = ', synset[0].attrib['sourceSense'])

Definition
language =  et
lecicon_sense =  s-kalapüük-n1


Next elements are relations between synsets spacified through target and relation type

In [45]:
print(synset[1].tag)
print('rel_confidence=', synset[1].attrib['confidenceScore'])
print('rel_type=', synset[1].attrib['relType'])
print('rel_status=', synset[1].attrib['status'])
print('rel_target=', synset[1].attrib['target'])


SynsetRelation
rel_confidence= 1.0
rel_type= similar
rel_status= unchecked
rel_target= estwn-et-34038-n


You can access these elements by specifying relative paths

In [160]:
definition = synset.xpath('Definition')
relations = synset.xpath('SynsetRelation')
print(len(definition),len(relations))

1 105


In [44]:
print(len(synsets))
print(len(entries))

NameError: name 'synsets' is not defined

In [90]:
print(entry[1].attrib)
print(synset[100].attrib)

{'synset': 'estwn-et-41417-n', 'id': 's-röntgen-n1', 'status': 'unchecked'}
{'target': 'estwn-et-71125-n', 'relType': 'hyponym', 'status': 'unchecked', 'confidenceScore': '1.0'}
